## Improve Performance & Submission

'모델 생성'까지는 베이스라인과 똑같이 실행합니다.

## 1. Fix the seed value and set up your GPU equipment (시드값 고정 및 GPU 장비 설정)

### [1] Fix the seed value (시드값 고정)

In [ ]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### [2] Set up GPU equipment (GPU 장비 설정)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

## 2. Prepare Data (데이터 준비)

In [ ]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### [1] Separate training/validation data (훈련/검증 데이터 분리)

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train,
                               test_size = 0.1,
                               stratify = train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                               random_state = 50)

### [2] Define dataset classes (데이터셋 클래스 정의)

In [ ]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform = None, is_test = False):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        # (1)
        self.is_test = is_test
    
    # 데이터셋 크기 반환 메서드
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.jpg' # (2) 이미지 파일 경로
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image = image)['image']
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃값도 반환
        if self.is_test: # (4)
            return image # (5) 테스트용일 때
        else:
            # (6) 타깃값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label # (7) 훈련/검증용일 때

### [3] Define image transformer (이미지 변환기 정의)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
transform_train = A.Compose([
    A.Resize(450, 650), # (1) 이미지 크기 조절
    A.RandomBrightnessContrast(brightness_limit = 0.2, # (2) 밝기 대비 조절
                               contrast_limit = 0.2, p = 0.3),
    A.VerticalFlip(p = 0.2),   # 상하 대칭 변환
    A.HorizontalFlip(p = 0.5), # 좌우 대칭 변환
    A.ShiftScaleRotate(        # (3) 이동, 스케일링, 회전 변환
        shift_limit = 0.1,
        scale_limit = 0.2,
        rotate_limit = 30, p = 0.3),
    A.OneOf([A.Emboss(p = 1), # (4) 양각화, 날카로움, 블러 효과
             A.Sharpen(p = 1),
             A.Blur(p = 1)], p = 0.3),
    A.PiecewiseAffine(p = 0.3), # (5) 어파인 변환
    A.Normalize(), # (6) 정규화 변환
    ToTensorV2()   # (7) 텐서로 변환
])

In [ ]:
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

### [4] Create datasets & a data loader (데이터셋 및 데이터 로더 생성)

In [ ]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir = img_dir, transform = transform_train)
dataset_valid = ImageDataset(valid, img_dir = img_dir, transform = transform_test)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size = batch_size,
                         shuffle = True, worker_init_fn = seed_worker,
                         generator = g, num_workers = 2)
loader_valid = DataLoader(dataset_valid, batch_size = batch_size,
                         shuffle = False, worker_init_fn = seed_worker,
                         generator = g, num_workers = 2)

## 3. Create Model (모델 생성)

In [ ]:
!pip install efficientnet-pytorch==0.7.1

In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes = 4) # (1)

model = model.to(device) # 장비 할당

## 4. Train Model & Validate Performance (모델 훈련 및 성능 검증)

손실 함수와 옵티마이저 설정도 베이스라인과 동일합니다. 그런 다음 스케줄러를 추가로 설정한 후 본격적인 훈련에 돌입하겠습니다.

### [1] Set Loss Function & Optimizer (손실 함수와 옵티마이저 설정)

In [ ]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.00006, weight_decay = 0.0001)

### [2] Set scheduler (스케줄러 설정)

스케줄러는 훈련 과정에서 학습률을 조정하는 기능을 제공합니다. 훈련 초반에는 빠르게 가중치를 갱신하기 위해서 학습률이 큰 게 좋습니다. 그러다가 훈련을 진행하면서 학습률을 점차 줄이면 최적 가중치를 찾기가 더 수월합니다.

골프를 생각하면 이해가 쉽습니다. 처음에는 공을 강하게 쳐서 멀리 날아가게 합니다. 홀과 가까울수록 약하게 쳐서 조금씩 움직이게 하죠.

여러 스케줄러가 있지만 여기서는 get_cosine_schedule_with_warmup() 스케줄러를 사용해보겠습니다. 지정한 값만큼 학습률을 증가시켰다가 코사인 그래프 모양으로 점차 감소시키는 스케줄러입니다.

In [ ]:
from transformers import get_cosine_schedule_with_warmup
epochs = 39 # (1) 총 에폭

scheduler = get_cosine_schedule_with_warmup(optimizer, # (2)
                                    num_warmup_steps = len(loader_train) * 3, # (3)
                                    num_training_steps = len(loader_train) * epochs) # (4)

(1) 베이스라인에서는 5였는데, 성능 개선을 위해 여기선 에폭 수를 39로 크게 늘렸습니다. 에폭이 너무 작으면 과소적합, 너무 많으면 과대적합이 일어나기 쉽습니다.

이어서 스케줄러를 생성했습니다. (2) 첫 번째 파라미터로 앞서 정의한 옵티마이저를 전달합니다. 그러면 해당 옵티마이저로 가중치를 갱신할 때 스케줄러로 학습률을 조정합니다. 참고로 학습률은 매 훈련 이터레이션마다 갱신됩니다.

(3) num_warmup_steps 파라미터는 몇 번만에 지정한 학습률(여기서는 0.00006)에 도달할지를 뜻합니다. 1에폭의 반복 수는 len(loader_train)입니다. 3에폭 만에 지정한 학습률에 도달하도록 len(loader_train) * 3을 전달했습니다. 총 반복 수(len(loader_train) * epoch)에 비해 상대적으로 작은 값이면 됩니다.

(4) num_training_steps 파라미터는 모든 훈련을 마치는 데 필요한 반복 횟수입니다. 총 39에폭만큼 훈련할 거니까 len(loader_train) * epochs를 전달했습니다.

### [3] Train & Validate Performance (훈련 및 성능 검증)

본격적으로 모델을 훈련시키며 성능을 검증해보겠습니다. 스케줄러 갱신 코드가 추가된 점만 빼면 모델 훈련 코드도 베이스라인과 완전히 같습니다.


In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대
epochs = 39

# 총 에폭만큼 반복
for epoch in range(epochs):
    # == [ 훈련 ] ==============================
    model.train()         # 모델을 훈련 상태로 설정
    epoch_train_loss = 0  # 에폭별 손실값 초기화 (훈련 데이터용)
    
    # '반복 횟수'만큼 반복
    for images, labels in tqdm(loader_train):
        # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 수차 (훈련 데이터용)
        epoch_train_loss += loss.item()
        loss.backward()  # 역전파 수행
        optimizer.step() # 가중치 갱신
        scheduler.step() ## 스케줄러 학습률 갱신
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    # == [ 검증 ] ==============================
    model.eval()          # 모델을 평가 상태로 설정
    epoch_valid_loss = 0  # 에폭별 손실값 초기화 (검증 데이터용)
    preds_list = []       # 예측 확률값 저장용 리스트 초기화
    true_onehot_list = [] # 실제 타깃값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            # 예측 확률값
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            # 실젯값 (원-핫 인코딩 형식)
            true_onehot = torch.eye(4, device = device)[labels].cpu().numpy()
            # 예측 확률값과 실젯값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
            
    # 검증 데이터 손실값 및 ROC AUC 점수 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')

학습을 시키는데, 약 5~6시간이 걸립니다. 검증 데이터 ROC AUC 값이 0.????이네요.

## 5. Predict & Submission (예측 및 제출)

기나긴 훈련을 마쳤습니다. 테스트 데이터를 활용해 예측한 뒤 제출할 일만 남았습니다. 이번에는 새로운 예측 기법을 배워보겠습니다. TTA와 레이블 스무딩 기법입니다.

### TTA(테스팅 단계 데이터 증강)

앞서 이미지 변환 패키지 albumentations를 활용해 훈련 데이터를 증강시켰습니다. 훈련 데이터가 많으면 모델 성능이 좋아집니다. 이러한 데이터 증강 기법을 테스트 단계에서도 이용하여 예측 성능을 더 끌어올릴 수 있습니다. 테스트 단계에서 활용하는 데이터 증강 기법을 TTA(Test-Time Augmentation)라고 합니다.

일반적으로 훈련된 모델이 테스트 데이터 원본을 활용해 타깃값을 예측합니다. 그런데 TTA를 적용하면 테스트 데이터를 여러 차례 변형한 뒤 예측합니다. 마치 테스트 데이터가 늘어난 효과를 얻습니다.

TTA를 활용해 예측하는 절차는 다음과 같습니다.
1. 테스트 데이터에 여러 변환을 적용합니다.
2. 변환된 테스트 데이터별로 타깃 확률값을 예측합니다.
3. 타깃 예측 확률의 평균을 구합니다.

3단계에서 구한 평균 확률을 최종 제출값으로 사용할 것입니다. 이렇게 하면 앙상블 효과가 있어서 원본 데이터로 한 차례만 예측할 때보다 성능이 좋아질 가능성이 높기 때문이죠.

In [ ]:
# 테스트 데이터 원본용 데이터셋 및 데이터 로더
dataset_test = ImageDataset(test, img_dir = img_dir,
                           transform = transform_test, is_test = True) # (1)

loader_test = DataLoader(dataset_test, batch_size = batch_size,
                         shuffle = False, worker_init_fn = seed_worker,
                        generator = g, num_workers = 2)


# TTA용 데이터셋 및 데이터 로더
dataset_TTA = ImageDataset(test, img_dir = img_dir,
                           transform = transform_train, is_test = True) # (2)

loader_TTA = DataLoader(dataset_test, batch_size = batch_size,
                         shuffle = False, worker_init_fn = seed_worker,
                        generator = g, num_workers = 2)

(1) 원본용 데이터셋을 만들 때 변환기로 transform_test를 전달했습니다. 원본용이므로 필수적인 변환만 가한 것입니다. 반면 TTA용에서는 transform_train을 전달하여 훈련 데이터처럼 여러 변환을 수행하도록 했습니다.

### [1] Predict (예측)

원본 테스트 데이터로 먼저 예측하고, 이어서 TTA를 적용해서도 예측해보겠습니다.

In [ ]:
model.eval() # 모델을 평가 상태로 설정

preds_test = np.zeros((len(test), 4)) # 예측값 저장용 배열 초기화

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률
        preds_part = torch.softmax(outputs.cpu(), dim = 1).squeeze().numpy()
        preds_test[i * batch_size:(i + 1) * batch_size] += preds_part

preds_test는 테스트 데이터 원본으로 예측한 타깃값입니다. 이 값을 'submission을 복사한 submission_test'에 저장해둡니다.

In [ ]:
submission_test = submission.copy() # 제출 샘플 파일 복사

submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_test

이어서 TTA를 적용해 예측해보겠습니다. TTA는 많이 할수록 앙상블 효과가 커지지만, 반복할수록 소요 시간 대비 효과가 미미해지므로 7번 정도면 충분합니다.

In [ ]:
num_TTA = 7

preds_tta = np.zeros((len(test), 4))

for i in range(num_TTA):
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            # 타깃 예측 확률
            preds_part = torch.softmax(outputs.cpu(), dim = 1).squeeze().numpy()
            preds_tta[i * batch_size:(i + 1) * batch_size] += preds_part

# TTA를 적용한 예측 확률 preds_tta의 평균
preds_tta /= num_TTA

예측한 타깃 확률을 submission_tta에 저장합니다.

In [ ]:
submission_tta = submission.copy()

submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta

### [2] Create Submission File (제출 파일 생성)

원본 테스트 데이터로 구한 예측값과 TTA를 적용해 구한 예측값을 각각 제출 파일로 만들겠습니다.

In [ ]:
submission_test.to_csv('submission_test.csv', index = False)
submission_tta.to_csv('submission_tta.csv', index = False)

### [3] Label Smoothing (레이블 스무딩)

간혹 딥러닝 모델이 과잉 확신하는 경우가 있습니다. 가량, 특정 타깃값일 확률을 1에 매우 가깝게 예측하는 경우입니다. 확신이 과하면 일반화 성능이 떨어질 우려가 있습니다. 즉, 최종 제출 시 평가 점수가 안 좋게 나올 수 있다는 뜻입니다. 그 확신이 올바르다면 다행이지만 항상 올바르게 예측하기는 어렵습니다. 일반화 성능을 높이려면 과잉 확신한 예측값을 보정해줘야 합니다. 이럴 때 사용하는 보정 기법이 레이블 스무딩(label smoothing)입니다.

In [ ]:
def apply_label_smoothing(df, target, alpha, threshold):
    # 타깃값 복사
    df_target = df[target].copy()
    k = len(target) # 타깃값 개수
    
    for idx, row in df_target.iterrows():        # (1)
        if (row > threshold).any():              # (2) 임곗값을 넘는 타깃값인지 판단
            row = (1 - alpha) * row + alpha / k  # (3) 레이블 스무딩 적용
            df_target.iloc[idx] = row            # (4) 레이블 스무딩을 적용한 값으로 변환
    return df_target # 레이블 스무딩을 적용한 타깃값 반환

각 파라미터의 의미는 다음과 같습니다.
- df : DataFrame
- target : 타깃값 이름의 리스트
- alpha : 레이블 스무딩 강도
- threshold : 레이블 스무딩을 적용할 최솟값 (타깃값이 임곗값을 넘을 때만 적용)

(1) 각 타깃값에 대해 (2) 임곗값을 넘는지 판단합니다. 임곗값을 넘으면 과잉 확신한 것이라고 간주해 레이블 스무딩을 적용하려는 겁니다.

(3) 레이블 스무딩을 적용합니다. (4) 그런 다음 현재 타깃값을 '레이블 스무딩을 적용한 값'으로 변환합니다.

In [ ]:
alpha = 0.001 # 레이블 스무딩 강도
threshold = 0.999 # 레이블 스무딩을 적용할 임곗값

# (1) 레이블 스무딩을 적용하기 위해 DataFrame 복사
submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

target = ['healthy', 'multiple_diseases', 'rust', 'scab'] # 타깃값 열 이름

# 레이블 스무딩 적용
submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target,
                                                  alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target,
                                                 alpha, threshold)

submission_test_ls.to_csv('submission_test_ls.csv', index = False)
submission_tta_ls.to_csv('submission_tta_ls.csv', index = False)

(1) 레이블 스무딩 적용 전후 결과를 모두 제출하기 위해 앞서 만든 submission_test와 submission_tta를 복사해 사용했습니다.

이상으로 제출 파일을 총 4개 만들었습니다.

- submission_test : 테스트 데이터 원본으로 예측
- submission_tta : TTA 적용
- submission_test_ls : submission_test에 레이블 스무딩 적용
- submission_tta_ls : submission_tta에 레이블 스무딩 적용

이제 다 끝났습니다. 

The End.